In [1]:
import pandas as pd
import crew_optimizer
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

In [3]:
assignment = pd.read_json("assignments/assignment.json")
assignment = assignment['assignment'][4:254]
assignment

D1_F026    {'captain': 'C26', 'first_officer': 'FO20', 'd...
D1_F043    {'captain': 'C44', 'first_officer': 'FO35', 'd...
D1_F005    {'captain': 'C34', 'first_officer': 'FO46', 'd...
D1_F023    {'captain': 'C6', 'first_officer': 'FO28', 'de...
D1_F031    {'captain': 'C32', 'first_officer': 'FO32', 'd...
                                 ...                        
D5_F042    {'captain': 'C31', 'first_officer': 'FO19', 'd...
D5_F003    {'captain': 'C36', 'first_officer': 'FO2', 'de...
D5_F001    {'captain': 'C35', 'first_officer': 'FO21', 'd...
D5_F021    {'captain': 'C45', 'first_officer': 'FO0', 'de...
D5_F023    {'captain': 'C21', 'first_officer': 'FO14', 'd...
Name: assignment, Length: 250, dtype: object

In [9]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
print(f"Using OpenAI API Key: {openai_api_key[:4]}...{openai_api_key[-4:]}")

Using OpenAI API Key: sk-A...dosP


In [6]:
instruction = f"""You are a large neighbourhood search (LNS) optimisation algorithm designer working
with an expert scheduler for an airline operation. Your task is to design a set of LNS destroy
and repair operators that will assign captains and first-officers to flights based on their
qualifications and availability. Each crew is certifications that determine which aircraft they can
fly. You must ensure that all shifts are adequately staffed while adhering to the following
constraints:"

1. Each flight must have one captain and one first-officer.
2. Captains can only be assigned to flights that match their certifications.
3. First-officers can only be assigned to flights that match their certifications.
4. Crew members cannot be assigned to overlapping flights.
5. Each crew member has a maximum number of hours they can work per week.
6. Try to balance the workload among crew members as evenly as possible.

Analyse the assignment data and propose a single destroy or repair operator that could be used
in the LNS algorithm to tackle the worst issue in the current assignment. Provide a detailed
description of how the operator works, including its logic and steps. Your response should be in
the following JSON format:

{{
  "operator_type": "destroy" or "repair",
  "description": "Detailed description of the operator logic and steps"
}}
"""

In [13]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": instruction},
        {
            "role": "user",
            "content": f"Here is the current assignment data: {assignment.to_json(orient='records')}",
        },
    ],
    temperature=0.7,
    max_tokens=1000,
    n=1,
    stop=None,
)

response_content = response.choices[0].message.content
print("Response from OpenAI:")
print(response_content)

#operator_suggestion = json.loads(response_content)
#print(operator_suggestion)

Response from OpenAI:
Based on the assignment data, one issue that strikes out is that some flights have either a captain or a first-officer missing. This suggests a problem with crew assignment. As a result, a repair operator can be designed to address this issue. The repair operator will focus on identifying flights with missing crew members and attempt to assign available, qualified crew members to these flights.

Here is a detailed description of the operator:

1. Identify flights with missing crew members: Check each flight in the schedule. If a flight does not have a captain or a first-officer, add it to the list of flights to be repaired.
2. Find available, qualified crew members: For each flight that needs to be repaired, check the availability and qualifications of each crew member. If a crew member is available during the flight time and has the required certification, add them to the list of potential assignments.
3. Assign crew members to flights: For each flight with poten